Import Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
np.random.seed(42)
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

Change 'data_dir'

In [2]:
data_dir = r"C:\Users\kyle_\OneDrive\Desktop\school_projects\456_project\archive\raw-img"
# Load images and labels
image_size = (128, 128)  # Target size for resizing
X = []
y = []
class_names = sorted(os.listdir(data_dir))  # Assumes one folder per class
class_indices = {name: idx for idx, name in enumerate(class_names)}

In [3]:
print("Loading images...")
for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            try:
                img = tf.keras.utils.load_img(image_path, target_size=image_size)  # Load and resize image
                img_array = tf.keras.utils.img_to_array(img)  # Convert to array
                X.append(img_array)
                y.append(class_indices[class_name])
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")

Loading images...


In [4]:
X = np.array(X)
y = np.array(y)
print(f"Loaded {len(X)} images.")

Loaded 26179 images.


In [5]:
# Normalize the images
X = X / 255.0

In [6]:
# Split data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Build a neural network model
cnn = tf.keras.models.Sequential()  # Initializing the CNN

# Convolutional Layers
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu", padding="same", input_shape=(128, 128, 3)))  # Convolution 1
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))  # Pooling 1
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"))  # Convolution 2
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"))  # Convolution 3
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))  # Pooling 2
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"))  # Convolution 4
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"))  # Convolution 5
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))  # Pooling 3

# Fully Connected Layers
cnn.add(tf.keras.layers.Flatten())  # Flattening
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))  # Fully Connected Layer 1
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=64, activation="relu"))  # Fully Connected Layer 2
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=len(class_names), activation="softmax"))  # Output Layer

C:\Users\kyle_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     8,388,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,513,802 (36.29 MB)

 Trainable params: 9,513,802 (36.29 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Compiling the Model
cnn.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [10]:
# Training the Model
history = cnn.fit(
    X_train, y_train,
    batch_size=32,
    epochs=5,
    validation_data=(X_valid, y_valid)
)

Epoch 1/5
655/655 ━━━━━━━━━━━━━━━━━━━━ 476s 719ms/step - accuracy: 0.1774 - loss: 2.2728 - val_accuracy: 0.1883 - val_loss: 2.2084
Epoch 2/5
655/655 ━━━━━━━━━━━━━━━━━━━━ 470s 717ms/step - accuracy: 0.1869 - loss: 2.2108 - val_accuracy: 0.1883 - val_loss: 2.2075
Epoch 3/5
555/655 ━━━━━━━━━━━━━━━━━━━━ 1:07 678ms/step - accuracy: 0.1847 - loss: 2.2119

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/sequential_1/conv2d_3_1/Relu defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a memory object, in file tensorflow/core/kernels/mkl/mkl_conv_ops.cc:1112
	 [[{{node StatefulPartitionedCall/sequential_1/conv2d_3_1/Relu}}]] [Op:__inference_multi_step_on_iterator_3088]